In [1]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from PIL import Image

In [2]:
tf.__version__

'2.10.0'

In [3]:
import os

In [4]:
rootdir = "./dataCNN/"
labels = []
for file in os.listdir(rootdir):
    labels.append(file)
    

In [50]:
labels

['1', '2']

In [51]:
x , y = [] , []

In [52]:
for index , label in enumerate(labels):
    path = rootdir + label
    for file in os.listdir(path):
        img = Image.open(os.path.join(path , file) , "r")
        img = np.asarray(img)
        x.append(img / 255)
        y.append(index)

In [53]:
len(x)

402

In [54]:
x[0]

array([[[0.19607843, 0.18431373, 0.11764706],
        [0.21176471, 0.2       , 0.14117647],
        [0.2       , 0.18431373, 0.14901961],
        ...,
        [1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843],
        [0.99607843, 0.99607843, 0.99607843]],

       [[0.18039216, 0.17254902, 0.11372549],
        [0.18823529, 0.18039216, 0.12941176],
        [0.18823529, 0.17647059, 0.14117647],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843]],

       [[0.15294118, 0.16078431, 0.10980392],
        [0.16078431, 0.16862745, 0.11764706],
        [0.17647059, 0.17647059, 0.1372549 ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       ...,

       [[0.0745098 , 0.08235294, 0.07843137],
        [0.0745098 , 0.08235294, 0.07843137],
        [0.08235294, 0

In [55]:
x = np.asarray(x)
y = np.asarray(y)

In [56]:
x.shape

(402, 256, 256, 3)

In [57]:
from functools import partial

defaultcon3d = partial(keras.layers.Conv2D , kernel_size = 3 , activation = 'relu' , padding = "SAME") 
model = keras.models.Sequential([
    defaultcon3d(kernel_size = 7 , filters = 64 , input_shape = [256,256,3]),
    keras.layers.MaxPooling2D(pool_size = 2),
    defaultcon3d(filters = 64),
    defaultcon3d(filters = 64),
    keras.layers.MaxPooling2D(pool_size = 2),
    defaultcon3d(filters = 128),
    defaultcon3d(filters = 128),
    keras.layers.MaxPooling2D(pool_size = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 150 , activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 100 , activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = len(labels) , activation = 'softmax')
])

In [58]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 256, 256, 64)      9472      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 128, 128, 64)      36928     
                                                                 
 conv2d_12 (Conv2D)          (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 128)      

In [59]:
model.compile(metrics = ["accuracy"] , loss = "sparse_categorical_crossentropy" , optimizer = "adam")

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
xtrain ,  xtest ,ytrain,  ytest = train_test_split(x , y , test_size = 0.2)

In [62]:
type(xtrain)

numpy.ndarray

In [63]:
xtrain.shape

(321, 256, 256, 3)

In [64]:
checkpoint = keras.callbacks.ModelCheckpoint('modelCNN.h5')
earlyStop = keras.callbacks.EarlyStopping(patience = 4 , restore_best_weights = True , monitor = 'accuracy')

In [65]:
history = model.fit(xtrain , ytrain , epochs = 10, callbacks = [checkpoint, earlyStop])

Epoch 1/10
11/11 [==============================] - 95s 8s/step - loss: 0.5577 - accuracy: 0.7352
Epoch 2/10
11/11 [==============================] - 88s 8s/step - loss: 0.1034 - accuracy: 0.9688
Epoch 3/10
11/11 [==============================] - 89s 8s/step - loss: 0.0163 - accuracy: 0.9938
Epoch 4/10
11/11 [==============================] - 89s 8s/step - loss: 0.0039 - accuracy: 1.0000
Epoch 5/10
11/11 [==============================] - 87s 8s/step - loss: 0.0136 - accuracy: 0.9969
Epoch 6/10
11/11 [==============================] - 87s 8s/step - loss: 0.0013 - accuracy: 1.0000
Epoch 7/10
11/11 [==============================] - 87s 8s/step - loss: 8.8936e-05 - accuracy: 1.0000
Epoch 8/10
11/11 [==============================] - 86s 8s/step - loss: 0.0000e+00 - accuracy: 1.0000


In [66]:
history

In [67]:
preds = model.predict(xtest)

3/3 [==============================] - 6s 2s/step


In [68]:
type(preds)

numpy.ndarray

In [69]:
ans = []
for pred in preds:
    ans.append(np.argmax(pred))

In [70]:
from sklearn.metrics import confusion_matrix

In [71]:
confusion_matrix(ans , ytest)

array([[38,  0],
       [ 0, 43]], dtype=int64)

In [72]:
ans

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1]

In [73]:
preds

array([[1.00000000e+00, 2.53752519e-10],
       [1.00000000e+00, 3.71604325e-09],
       [1.00000000e+00, 3.76815662e-11],
       [2.84123717e-14, 1.00000000e+00],
       [1.00000000e+00, 3.03770169e-11],
       [1.00000000e+00, 3.35823799e-11],
       [1.00712543e-13, 1.00000000e+00],
       [1.35119932e-12, 1.00000000e+00],
       [1.00000000e+00, 3.48424567e-10],
       [1.00000000e+00, 3.29315325e-10],
       [1.47223576e-14, 1.00000000e+00],
       [2.46342033e-14, 1.00000000e+00],
       [4.69358656e-13, 1.00000000e+00],
       [1.00000000e+00, 3.48424567e-10],
       [1.87170649e-14, 1.00000000e+00],
       [1.00000000e+00, 3.23018313e-11],
       [1.00000000e+00, 8.14810353e-09],
       [3.70985014e-11, 1.00000000e+00],
       [3.42079143e-11, 1.00000000e+00],
       [1.00000000e+00, 7.42470796e-09],
       [1.00000000e+00, 3.48424567e-10],
       [2.33873420e-14, 1.00000000e+00],
       [1.09269900e-13, 1.00000000e+00],
       [4.11425373e-11, 1.00000000e+00],
       [3.245017